In [1]:
import alpaca_trade_api as tradeapi
import time
import threading

# Alpaca API credentials
API_KEY = 'PKL0QPAXI2XEQN42OD2S'
API_SECRET = 'nkqziIUvRn7rkEihNRVpw9IcNYftpDxVHPRQnyS3'
BASE_URL = 'https://paper-api.alpaca.markets'

# Initialize API
api = tradeapi.REST(API_KEY, API_SECRET, base_url=BASE_URL)

# News data storage
all_news = []

# List of tickers
tickers = [
    "AAPL", "MSFT", "GOOGL", "AMZN", "FB", "INTC", "TSLA", "NVDA", "ORCL", "IBM",
    "JPM", "BAC", "WFC", "C", "GS", "AXP", "MS", "BRK.B",
    "JNJ", "PFE", "UNH", "MRK", "ABBV", "TMO", "ABT",
    "PG", "KO", "PEP", "NKE", "MO",
    "GE", "MMM", "BA", "HON", "CAT",
    "XOM", "CVX", "BP", "TOT",
    "NEP", "DUK", "SO",
    "T", "VZ",
    "DIS", "MCD", "SBUX",
    "WMT", "HD"
]


# Function to fetch news for a ticker
def fetch_news(ticker):
    global all_news
    try:
        news = api.get_news(symbol=ticker, limit=200)  # Adjust the limit as needed
        
        headline = clean_text(article.headline)
        summary = clean_text(article.summary)
        for article in news:
            all_news.append({
                'headline': headline,
                'summary': summary,
                'created_at': extract_date(article.created_at),
                'sentiment': get_sentiment(headline, summary)
            })
    except Exception as e:
        print(f"Error fetching news for {ticker}: {e}")

# Function to process requests in batches
def process_requests():
    while tickers:
        threads = []
        for _ in range(200):  # 200 parallel threads, respecting the rate limit
            if tickers:
                ticker = tickers.pop(0)
                thread = threading.Thread(target=fetch_news, args=(ticker,))
                threads.append(thread)
                thread.start()

        # Wait for all threads to complete
        for thread in threads:
            thread.join()

        # Wait for 60 seconds before the next batch
        time.sleep(60)

# Start processing
process_requests()


In [2]:
import csv

# Define CSV file name
csv_file = 'financial_news.csv'

# Write data to CSV
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=all_news[0].keys())
    writer.writeheader()
    for news_item in all_news:
        writer.writerow(news_item)

print(f'Data saved to {csv_file}')

Data saved to financial_news.csv


In [5]:
import pandas as pd

df = pd.read_csv(csv_file)
df

,headline,summary,created_at,url
0,Meta Blocks Adults From Messaging Teens They D...,Meta Platforms has unveiled stricter messaging...,2024-01-25 13:37:00+00:00,https://www.benzinga.com/news/24/01/36779229/m...
1,"After 130% Surge, AMD Faces Uncertain Road Ahe...",AMD&#39;s stock surged in 2023 due to high dem...,2024-01-23 08:03:02+00:00,https://www.benzinga.com/analyst-ratings/analy...
2,Legendary Investor Jim Rogers Predicts Market ...,Rogers has identified indicators of an impendi...,2023-12-29 09:11:30+00:00,https://www.benzinga.com/news/23/12/36423331/l...
3,Mark Zuckerberg's Meta Said To Be Worried Abou...,This story was first published on the Benzinga...,2023-12-21 08:26:56+00:00,https://www.benzinga.com/government/23/12/3633...
4,"Now, Facebook And Messenger Also Get End-To-En...",Meta&#39;s new encryption feature implies that...,2023-12-07 11:08:57+00:00,https://www.benzinga.com/news/23/12/36129319/n...
...,...,...,...,...
9775,Investor Optimism Improves After US Stocks Set...,The CNN Money Fear and Greed index showed some...,2023-04-24 06:33:38+00:00,https://www.benzinga.com/news/earnings/23/04/3...
9776,Coca-Cola's Earnings Outlook,,2023-04-21 14:00:37+00:00,https://www.benzinga.com/news/earnings/23/04/3...
9777,Benzinga Pro's Top 10 Stocks To Watch For Frid...,\n\t\n\t\t\n\t\t\t\n\t\t\tThe next two weeks w...,2023-04-21 12:59:53+00:00,https://www.benzinga.com/pre-market-outlook/23...
9778,"AI Rollout, Plus New Partnerships For Snap: Wh...",Snap Inc (NYSE SNAP) shares are trading higher...,2023-04-19 18:51:19+00:00,https://www.benzinga.com/news/23/04/31899478/a...


In [15]:
import re
import pandas as pd

# Keep digits and symbols that can be significant in financial analysis. 
# E.g. digits could represent financial figures, and symbols like '$' might indicate currency amounts. 
def clean_text(text):
    # Remove URLs
    text = re.sub(r'http\S+', '', text)  
    # Remove newlines and tabs
    text = text.replace('\n', ' ').replace('\r', '').replace('\t', ' ') 
    # Keep words, digits, $ and .
    text = re.sub(r'[^\w\d$.]', ' ', text)  
    # Replace multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)  
    return text.strip()

In [16]:
'''
Testing clean_data()
'''
# Example DataFrame
data = pd.DataFrame({
    'article': ["Stocks rallied, with AAPL reaching $150.50. More at http://example.com", 
                "Q1 profits fell to $1.5 billion, down from $2 billion. Visit http://finance.com for more.", 
                "Tech stocks, e.g., MSFT, AMZN, and GOOG, showed mixed results today."]
})

data['cleaned_article'] = data['article'].apply(clean_text)
print(data)

                                             article  \
0  Stocks rallied, with AAPL reaching $150.50. Mo...   
1  Q1 profits fell to $1.5 billion, down from $2 ...   
2  Tech stocks, e.g., MSFT, AMZN, and GOOG, showe...   

                                     cleaned_article  
0  Stocks rallied with AAPL reaching $150.50. Mor...  
1  Q1 profits fell to $1.5 billion down from $2 b...  
2  Tech stocks e.g. MSFT AMZN and GOOG showed mix...  


In [13]:
from datetime import datetime

def extract_date(date_str):
    try:
        # Parse the string to a datetime object
        dt = datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S%z')
        # Format the datetime object to keep only the date part
        return dt.strftime('%Y-%m-%d')
    except ValueError as e:
        print(f"Error parsing date: {e}")
        return None

2024-01-25


In [ ]:
'''
Testing extract_date()
'''

date_string = "2024-01-25 13:37:00+00:00"
extracted_date = extract_date(date_string)
print(extracted_date)

In [1]:
from transformers import pipeline

classifier = pipeline('sentiment-analysis')

def get_sentiment(headline, summary):
    sentiment = classifier(summary + headline)
    
    if sentiment['label'] == 'POSITIVE':
        return f"{sentiment['score']}.2f"

    return f"-{sentiment['score']}.2f"

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


D:\Programs\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\scsar\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [17]:
# Example usage
cleaned_text = clean_text("Example Text! With numbers 123 and punctuation.")

from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')  # Download the required NLTK models

def tokenize(text):
    return word_tokenize(text)

# Example usage
tokens = tokenize(cleaned_text)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\scsar\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
